In [64]:
import polars as pl
from pathlib import *

In [5]:
# Setting up paths

In [65]:
base = Path(".")
input = base / "data" / "input"
traits_path = input / "traits_no_measurement.tsv"
traits_df = pl.read_csv(str(traits_path), sep="\t")

traits_df_nullsrem = (
    traits_df
        .with_column(pl.all().fill_null(0))
        .filter(pl.col("Olga's comments")!="0")
) #removed nulls

In [66]:
comments = traits_df_nullsrem["Olga's comments"].to_list()
comments_low = [x.lower().strip() for x in comments]
comments_series = pl.Series(comments_low)

traits_grouped = traits_df_nullsrem.with_columns(pl.lit(comments_series).alias("comments")).drop("Olga's comments").groupby('comments').agg(pl.all())


In [67]:
columns = traits_grouped.drop(["comments"]).columns

traits = traits_grouped.explode(columns)
traits

comments,trait,trait_uri,studies,association_counts,papers
str,str,str,i64,i64,i64
"""glucose homeos...","""type 2 diabete...","""http://purl.ob...",175,5003,175
"""glucose homeos...","""body mass inde...","""http://www.ebi...",151,8912,151
"""glucose homeos...","""fat body mass""","""http://www.ebi...",40,184,40
"""glucose homeos...","""obesity""","""http://www.ebi...",38,196,38
"""glucose homeos...","""waist circumfe...","""http://www.ebi...",31,500,31
"""glucose homeos...","""body fat perce...","""http://www.ebi...",23,527,23
"""glucose homeos...","""metabolic synd...","""http://www.ebi...",22,233,22
"""glucose homeos...","""non-alcoholic ...","""http://www.ebi...",22,169,22
"""glucose homeos...","""insulin resist...","""http://www.ebi...",12,21,12


In [63]:
modules = set(comments_low)
df_list = []

dir_path = base / "data" / "interim" 

for item in modules:
     # creating directory
    if "/" in item:
        item = item.replace("/", ", ")
    if "?" in item:
        item = item.replace("?", "")
                            
    item_path = os.path.join(dir_path, item)
    os.makedirs(item_path,exist_ok=True)
    file_name = item + '.tsv'
    file_path = dir_path / item / file_name

    module_df = traits.filter(pl.col('comments')==item)

    module_df.write_csv(file = str(file_path), sep='\t')
